### Student Information
Name: 盧子涵

Student ID: 113065542

GitHub ID: luzi2023

Kaggle name: 

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__. 

In [2]:
### Begin Assignment Here

### =======================Second Part=================================

In [1]:
import pandas as pd
import numpy as np
import json
import emoji
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import transformers

In [2]:
raw_tweet = "tweets_DM.json"
data = []
with open(raw_tweet, "r") as f_in:
    for line in f_in:
        data.append(json.loads(line))

data_identification = pd.read_csv("./data_identification.csv")
data_identification.head(3)

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train


In [3]:
# 定義表情符號到關鍵字的映射字典
emoji_dict = {
    '😂': '[joy]',
    '❤️': '[love]',
    '😍': '[adoration]',
    '😭': '[cry]',
    '❤': '[care]',
    '😊': '[happy]',
    '🙏': '[pray]',
    '😘': '[kiss]',
    '💕': '[love_each_other]',
    '🔥': '[fire]',
    '😩': '[weary]',
    '🤔': '[think]',
    '💯': '[perfect]',
    '💙': '[loyalty]',
    '🙄': '[annoyed]',
    '😁': '[happy]',
    '🙌': '[celebrate]',
    '🙏🏾': '[pray]',
    '👍': '[approve]',
    '🙏🏽': '[pray]'
}

# 定義清理推文文本的函數
def clean_tweet(text, emoji_dict):
    # 將定義的表情符號替換為對應的關鍵詞
    for emj, keyword in emoji_dict.items():
        text = text.replace(emj, keyword)
    # 移除其餘的表情符號
    text = emoji.replace_emoji(text, replace='')
    # 移除 <LH> 標籤
    text = re.sub(r'<LH>', '', text)
    # 移除多餘的空白字元
    text = text.strip()
    return text

In [4]:
processed_tweets = []
for entry in data:
    # 檢查 '_source' 和 'tweet' 是否存在於記錄中
    if '_source' in entry and 'tweet' in entry['_source']:
        tweet = entry['_source']['tweet']
        # 檢查 'text' 是否存在於 'tweet' 中
        if 'text' in tweet:
            tweet_text = tweet['text']
            cleaned_text = clean_tweet(tweet_text, emoji_dict)
            # 創建處理後的推文記錄，保留 '_source' 和 'tweet'
            processed_tweet = {
                '_source': {
                    'tweet': tweet.copy()
                }
            }
            # 更新清理後的文本
            processed_tweet['_source']['tweet']['text'] = cleaned_text
            processed_tweets.append(processed_tweet)
        else:
            print("記錄中缺少 'text' 鍵")
    else:
        print("記錄中缺少 '_source' 或 'tweet' 鍵")

# 將處理後的推文資料存儲為 JSON 檔案
with open('tweets_DM_filtered_1.json', 'w', encoding='utf-8') as outfile:
    json.dump(processed_tweets, outfile, ensure_ascii=False, indent=4)

In [5]:
# 將處理後的資料轉換為 DataFrame
df_processed = pd.DataFrame(processed_tweets)

df = pd.DataFrame(data)
_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})

In [6]:
df = df.merge(data_identification, on='tweet_id', how='left')

In [7]:
emotions = pd.read_csv("emotion.csv")
emotions['emotion'].value_counts()

emotion
joy             516017
anticipation    248935
trust           205478
sadness         193437
disgust         139101
fear             63999
surprise         48729
anger            39867
Name: count, dtype: int64

In [8]:
train_data = df[df['identification'] == 'train']
test_data = df[df['identification'] == 'test']
train_data = train_data.merge(emotions, on='tweet_id', how='left')
train_data.drop_duplicates(subset=['text'], keep=False, inplace=True)

# 資料樣本化
train_data_sample = train_data.sample(frac=0.3, random_state=42)
y_train_data = train_data_sample['emotion']
X_train_data = train_data_sample['text']

# 標籤編碼
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train_data)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_train_data, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [20]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_length = 100
tokenized_train = tokenizer(X_train.tolist(), max_length=max_length, truncation=True, padding=True, return_tensors="tf")
tokenized_test = tokenizer(X_test.tolist(), max_length=max_length, truncation=True, padding=True, return_tensors="tf")

In [21]:
tokenized_train[:1]

{'input_ids': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
 array([[  101,  2006,  1030,  9130,  1030, 16021, 23091,  1030, 10474,
          4385,  2057,  2035,  1026,  1048,  2232,  1028,  1001,  2183,
          1026,  1048,  2232,  1028,  2003,  1001,  5533,  2011,  1001,
          2372,  2092,  1045,  1001,  7714,  1001,  2123,  1005,  1056,
          1026,  1048,  2232,  1028,  2009,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]])>,
 'token_type_ids': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [42]:
from tensorflow.keras import layers
from transformers import TFBertModel
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
import keras.saving

# class BertClassifier(tf.keras.Model):
#     def __init__(self, num_classes):
#         super(BertClassifier, self).__init__()
#         self.bert = TFBertModel.from_pretrained("bert-base-uncased")
#         self.classifier = layers.Dense(num_classes, activation='softmax')  # num_classes 為分類數量

#     def call(self, inputs):
#         # BERT 的輸出包含 last_hidden_state 和 pooler_output
#         outputs = self.bert(inputs)
#         pooled_output = outputs.pooler_output  # [CLS] token 的輸出
#         logits = self.classifier(pooled_output)
#         return logits

@keras.saving.register_keras_serializable()  # 註冊類以供保存和載入
class BertClassifier(Model):
    def __init__(self, num_classes, model_name="bert-base-uncased"):
        super(BertClassifier, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name)
        self.classifier = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        outputs = self.bert(**inputs)
        pooled_output = outputs.pooler_output
        return self.classifier(pooled_output)


In [36]:
num_classes = 8
model = BertClassifier(num_classes)

input_ids = tokenized_train['input_ids']
attention_mask = tokenized_train['attention_mask']

train_input = {'input_ids': input_ids, 'attention_mask': attention_mask}

# logits = model({'input_ids': input_ids, 'attention_mask': attention_mask})
# print(logits)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_input, y_train, epochs=3, batch_size=32)

model.save("bert_classifier_model.keras")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

 5293/10869 ━━━━━━━━━━━━━━━━━━━━ 14:43:45 10s/step - accuracy: 0.3855 - loss: 1.6899

KeyboardInterrupt: 

In [43]:
from tensorflow.keras.models import load_model
from transformers import TFBertForSequenceClassification

# 載入模型
loaded_model = load_model("bert_classifier_model.keras")

# 使用加載的模型進行推理
predictions = loaded_model.predict(train_inputs)


TypeError: Could not locate class 'BertClassifier'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'BertClassifier', 'config': {'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'BertClassifier', 'build_config': {'input_shape': {'input_ids': [None, 100], 'attention_mask': [None, 100]}}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 0.0010000000474974513, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name': None}, 'loss': 'sparse_categorical_crossentropy', 'l